In [39]:
import numpy as np
import pandas as pd
from sklearn import svm

from ipynb.fs.full.ECE610_HW2 import pca_analysis

### Part 1 Logistic Regression

In [40]:
# Part 1 Logistic Regression

class logistic_regression(object):
    
    def __init__(self,lr = 0.001,iterations = 1000):
        
        # Initializae running parameters
        self.iterations = iterations
        self.lr = lr
        
    def fit(self,data): # Data has labels in first column
                        # Data is a pandas dataframe
        
        n_samples = data.shape[0] # Number of samples in dataset
        
        # Augment feature vector to include constant bias term
        bias = np.ones((n_samples,1)).reshape(n_samples,1) # Add Bias
        
        # Add bias to matrix of 
        try:
            data.insert(1,'bias',bias)
        except:
            pass
        
        # Convert to numpy array
        data = data.to_numpy()
        
        # format to M x N matrix
        data = data.transpose()

        # Seperate Labels and Features
        labels = data[0,:]
        features = data[1:,:]
        
        # Number of features
        n_inputs = features.shape[0]

        # Init random weight vector
        self.weights = np.random.randn(n_inputs,1).reshape(n_inputs,1)
        
        # Init counter for iterations 
        count = 0
        
        # Add epsilon for convergence 
        epsilon = np.array([self.lr/1000]*n_inputs).reshape(n_inputs,1)
        
        # Perform iterations
        while count < self.iterations:
            
            # Predict class using current weights
            yPred = sigmoid(np.matmul(self.weights[1:].transpose(),features[1:,:]) + self.weights[0])
            
            # compute gradient
            gradient = np.matmul((yPred-labels),features.transpose())
            gradient = gradient.reshape((n_inputs,1))
            
            # Store previously computed weight
            self.weights0 = self.weights
            
            # Update Weights
            self.weights = self.weights - self.lr*gradient

            # Check for convergence
            if (np.abs(self.weights - self.weights0) < epsilon).all():
                print("Total iterations ran: {}".format(count))
                break
            
            count += 1
    
        return self.weights
    
    def test(self,data):
        
        # Convert to array of M x N with labels in first row
        data = data.to_numpy().transpose()
        
        # N test samples
        n_test = data.shape[1]
        
        # Seperate labels and inputs
        target = data[0,:]
        x = data[1:,:]
        
        # Predict Probability of Class 1
        yPred = sigmoid(np.matmul(self.weights[1:].transpose(),x) + self.weights[0])
        
        # Assign Class label
        classification = yPred.round()
        
        # Calculate classification accuracy
        misclassified = np.abs(classification - target)
        acc = (n_test-misclassified.sum())/n_test

        print("Classification Accuracy: {:.2%}".format(acc))
        
        return yPred
    
def sigmoid(a):
    """ Sigmoid Function """
    return 1./(1.+np.exp(-a))
    


In [41]:
# Import TwoClassTrain and TwoClassTest data
train_df_data = pd.read_csv("TwoClassTrain.csv",header = None, index_col = False)
test_df_data = pd.read_csv("TwoClassTest.csv",header = None, index_col = False)

In [42]:
# Train logistic regression classifier for TwoClassTrain data
net = logistic_regression(lr = .00005,iterations = 10000)
weights = net.fit(train_df_data)
print(weights)

# Run Test data
yPred = net.test(test_df_data)

Total iterations ran: 2801
[[ 1.27291967]
 [-0.60316211]
 [ 0.10441946]]
Classification Accuracy: 79.25%


In [43]:
# Import MNIST2Class data
data = pd.read_csv("MNIST2class.csv",header = None, index_col = False)

In [44]:
# Set class labels to 0 and 1
data.loc[data[0] == 4,0] = 0
data.loc[data[0] == 8,0] = 1

# N test samples
n = data.shape[0]

# Run PCA algorithm for feature generation
pca = pca_analysis(data)
mean_vec, w, v, sorted_index = pca.solve()
score = pca.proj()
score = np.real(score)
score = score.transpose()
data = data.to_numpy()

# Seperate labels 
labels = data[:,0].reshape(n,1)

# Combine Labels with 1st and 2nd Principal Component Projections
data = np.concatenate((labels,score),axis = 1)

# Seperate Training Data
train = np.concatenate((data[data[:,0] == 0][:800,:3],data[data[:,0] == 1][:800,:3]))

train_df_MNIST = pd.DataFrame(train)

X_train_MNIST = train_df_MNIST.to_numpy()[:,1:]
y_train_MNIST = train_df_MNIST.to_numpy()[:,0].ravel()

# Seperate Test Data
test = np.concatenate((data[data[:,0] == 0][800:,:3],data[data[:,0] == 1][800:,:3]))

test_df_MNIST = pd.DataFrame(test)

X_test_MNIST = test_df_MNIST.to_numpy()[:,1:]
y_test_MNIST = test_df_MNIST.to_numpy()[:,0].ravel()


In [45]:
# Train logistic regression classifier for MNIST Data
net = logistic_regression(lr = 0.01,iterations = 10000)
weights = net.fit(train_df_MNIST)
print(weights)
yPred = net.test(test_df_MNIST)


[[ 1716.04487538]
 [-2555.10192338]
 [-1364.7718019 ]]
Classification Accuracy: 91.50%


### Part 2 Support Vector Machines

In [46]:
from sklearn import svm

# Seperate Training data and labels
X_train = train_df_data[[1,2]].to_numpy()
y_train = train_df_data[[0]].to_numpy().ravel()

# Seperate Test data and labels
X_test = test_df_data[[1,2]].to_numpy()
y_test = test_df_data[[0]].to_numpy().ravel()

In [47]:
def test_accuracy(target, prediction): #Inputs are columns
    
    """Calculate Accuracy of classifier using target labels and predictions"""
    n_points = target.shape[0]
    misclassified = np.abs(target - prediction)

    return (n_points - misclassified.sum())/n_points

In [48]:
# Init SVM model for linear, polynomial, and radial basis function
linear_svc = svm.SVC(kernel = 'linear')
poly_svc = svm.SVC(kernel = 'poly')
rbf_svc = svm.SVC(kernel = 'rbf')

# Fit model to data
linear_svc.fit(X_train,y_train)
poly_svc.fit(X_train,y_train)
rbf_svc.fit(X_train,y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [49]:
# Predict class of test data
yPred_linear = linear_svc.predict(X_test)
acc_linear = test_accuracy(y_test,yPred_linear)

yPred_poly = poly_svc.predict(X_test)
acc_poly = test_accuracy(y_test,yPred_poly)

yPred_rbf = rbf_svc.predict(X_test)
acc_rbf = test_accuracy(y_test,yPred_poly)

print("SVM Linear Kernel Acc: {:.2%}".format(acc_linear))
print("SVM Polynomial Kernel Acc: {:.2%}".format(acc_poly))
print("SVM Radial Basis Function Kernel Acc: {:.2%}".format(acc_rbf))

SVM Linear Kernel Acc: 77.50%
SVM Polynomial Kernel Acc: 99.50%
SVM Radial Basis Function Kernel Acc: 99.50%


In [50]:
# MNIST Data

# Init SVM model for linear, polynomial, and radial basis function
linear_svc_MNIST = svm.SVC(kernel = 'linear')
poly_svc_MNIST = svm.SVC(kernel = 'poly')
rbf_svc_MNIST = svm.SVC(kernel = 'rbf',gamma = 1/4)

linear_svc_MNIST.fit(X_train_MNIST,y_train_MNIST)
poly_svc_MNIST.fit(X_train_MNIST,y_train_MNIST)
rbf_svc_MNIST.fit(X_train_MNIST,y_train_MNIST)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.25, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [51]:
# Predict class of test data
yPred_linear_MNIST = linear_svc_MNIST.predict(X_test_MNIST)
acc_linear_MNIST = test_accuracy(y_test_MNIST,yPred_linear_MNIST)

yPred_poly_MNIST = poly_svc_MNIST.predict(X_test_MNIST)
acc_poly_MNIST = test_accuracy(y_test_MNIST,yPred_poly_MNIST)

yPred_rbf_MNIST = rbf_svc_MNIST.predict(X_test_MNIST)
acc_rbf_MNIST = test_accuracy(y_test_MNIST,yPred_poly_MNIST)

print("SVM Linear Kernel Acc: {:.2%}".format(acc_linear_MNIST))
print("SVM Polynomial Kernel Acc: {:.2%}".format(acc_poly_MNIST))
print("SVM Radial Basis Function Kernel Acc: {:.2%}".format(acc_rbf_MNIST))

SVM Linear Kernel Acc: 91.75%
SVM Polynomial Kernel Acc: 89.25%
SVM Radial Basis Function Kernel Acc: 89.25%


In [52]:
# Train & Test Polynomial SVM with Degree=1

poly_svc_MNIST = svm.SVC(kernel = 'poly',degree = 1)
poly_svc_MNIST.fit(X_train_MNIST,y_train_MNIST)
yPred_poly_MNIST = poly_svc_MNIST.predict(X_test_MNIST)
acc_poly_MNIST = test_accuracy(y_test_MNIST,yPred_poly_MNIST)
print("SVM Polynomial Kernel Degree=1 Acc: {:.2%}".format(acc_poly_MNIST))

SVM Polynomial Kernel Degree=1 Acc: 92.25%
